In [1]:
# preprocessing from data_processing.ipynb
import pandas as pd
import numpy as np
import random

# Set seed for any random operations
random.seed(1)

# First, let's open the file!
print("loading...")
data = pd.read_excel("SBA_Loan_data_.xlsx", "Sheet1")
# microEcon = pd.read_csv("SBA_Macroecon.csv")
print("loaded")

loading...
loaded


In [2]:
print data.shape
data.head(5)

(147423, 30)


,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,CDC_Name,CDC_Street,CDC_City,CDC_State,...,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,BusinessType,LoanStatus,ChargeOffDate,GrossChargeOffAmount
0,504,ROBERT G. FIELDS AND MARY D. F,55 & RICHLAND,KANSAS CITY,KS,66106,Avenue Area Incorporated,3324 Emerald Lane,Jefferson City,MO,...,NaN,12,NaN,NaN,WYANDOTTE,KS,INDIVIDUAL,PIF,NaT,0.0
1,504,CANDLELAMP COMPANY,3454 NIKI WAY,RIVERSIDE,CA,92507,CDC Small Business Finance Cor,2448 Historic Decatur,San Diego,CA,...,NaN,240,NaN,NaN,RIVERSIDE,CA,INDIVIDUAL,PIF,NaT,0.0
2,504,"NATIONAL COMPOSITES, INC.",2303 N. BENDIX DRIVE,SOUTH BEND,IN,46628,Business Development Corporati,218 W. Washington Street,South Bend,IN,...,NaN,120,NaN,NaN,ST JOSEPH,IN,CORPORATION,PIF,NaT,0.0
3,504,WENDY'S OF HARRISBURG,OUTLOT A. TR. 45N & SMALL ST,HARRISBURG,IL,62946,Small Business Growth Corporat,2401 West White Oaks Drive,Springfield,IL,...,NaN,240,NaN,NaN,SALINE,IL,CORPORATION,CHGOFF,2003-03-28,0.0
4,504,"RON SAUNORIS GARDEN CTR, INC.",13747 W. 159TH ST.,LOCKPORT,IL,60441,MISSNG/INACTV LENDER,NaN,NaN,NaN,...,NaN,240,NaN,NaN,WILL,IL,CORPORATION,CANCLD,NaT,0.0


In [4]:
# print microEcon.shape
# microEcon.head(5)

In [5]:
# Drop bad data
data = data[data.LoanStatus != 'CANCLD']
data = data[data.LoanStatus != 'EXEMPT']

# Drop NaN rows found previously
# data = data.dropna(subset=['BorrState'])

# Specify that NAICS + ZIP codes and 'ApprovalFiscalYear' are categorical
# data[['NaicsCode','BorrZip','CDC_Zip','ApprovalFiscalYear']] = data[['NaicsCode','BorrZip','CDC_Zip','ApprovalFiscalYear']].fillna("MISSING")
# data[['NaicsCode','BorrZip','CDC_Zip','ApprovalFiscalYear']] = data[['NaicsCode','BorrZip','CDC_Zip','ApprovalFiscalYear']].astype(str)

# Fill in the empty entries under 'ChargeOffDate' with NA/not applicable
# data.update(data['ChargeOffDate'].fillna("NA"))

data.head()

,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,CDC_Name,CDC_Street,CDC_City,CDC_State,...,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,BusinessType,LoanStatus,ChargeOffDate,GrossChargeOffAmount
0,504,ROBERT G. FIELDS AND MARY D. F,55 & RICHLAND,KANSAS CITY,KS,66106,Avenue Area Incorporated,3324 Emerald Lane,Jefferson City,MO,...,NaN,12,NaN,NaN,WYANDOTTE,KS,INDIVIDUAL,PIF,NaT,0.0
1,504,CANDLELAMP COMPANY,3454 NIKI WAY,RIVERSIDE,CA,92507,CDC Small Business Finance Cor,2448 Historic Decatur,San Diego,CA,...,NaN,240,NaN,NaN,RIVERSIDE,CA,INDIVIDUAL,PIF,NaT,0.0
2,504,"NATIONAL COMPOSITES, INC.",2303 N. BENDIX DRIVE,SOUTH BEND,IN,46628,Business Development Corporati,218 W. Washington Street,South Bend,IN,...,NaN,120,NaN,NaN,ST JOSEPH,IN,CORPORATION,PIF,NaT,0.0
3,504,WENDY'S OF HARRISBURG,OUTLOT A. TR. 45N & SMALL ST,HARRISBURG,IL,62946,Small Business Growth Corporat,2401 West White Oaks Drive,Springfield,IL,...,NaN,240,NaN,NaN,SALINE,IL,CORPORATION,CHGOFF,2003-03-28,0.0
6,504,"ROCKY MTN EMPLOYE BENEFTS, INC",3200 S. 700 E.,SALT LAKE CITY,UT,84106,Mountain West Small Business F,2595 East 3300 South,Salt Lake City,UT,...,NaN,240,NaN,NaN,SALT LAKE,UT,CORPORATION,PIF,NaT,0.0


In [6]:
data.shape
list(data)

[u'Program',
 u'BorrName',
 u'BorrStreet',
 u'BorrCity',
 u'BorrState',
 u'BorrZip',
 u'CDC_Name',
 u'CDC_Street',
 u'CDC_City',
 u'CDC_State',
 u'CDC_Zip',
 u'ThirdPartyLender_Name',
 u'ThirdPartyLender_City',
 u'ThirdPartyLender_State',
 u'ThirdPartyDollars',
 u'GrossApproval',
 u'ApprovalDate',
 u'ApprovalFiscalYear',
 u'DeliveryMethod',
 u'subpgmdesc',
 u'InitialInterestRate',
 u'TermInMonths',
 u'NaicsCode',
 u'NaicsDescription',
 u'ProjectCounty',
 u'ProjectState',
 u'BusinessType',
 u'LoanStatus',
 u'ChargeOffDate',
 u'GrossChargeOffAmount']

In [7]:
from scipy.sparse import csr_matrix
from sklearn.cross_validation import train_test_split


newdf = data[['BorrCity', 'BorrState', 'BorrZip', 'GrossApproval',\
              'ApprovalDate','ApprovalFiscalYear', 'TermInMonths', \
              'ProjectCounty', 'BusinessType', 'LoanStatus']]
# drop nan rows

/Users/hujiayu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
newdf = newdf.dropna(subset = ['LoanStatus'])
newdf = newdf.dropna(axis=0, how='any')
print newdf.shape
newdf.head(20)

(54719, 10)


,BorrCity,BorrState,BorrZip,GrossApproval,ApprovalDate,ApprovalFiscalYear,TermInMonths,ProjectCounty,BusinessType,LoanStatus
0,KANSAS CITY,KS,66106,166000,1990-01-02,1990,12,WYANDOTTE,INDIVIDUAL,PIF
1,RIVERSIDE,CA,92507,117000,1990-01-02,1990,240,RIVERSIDE,INDIVIDUAL,PIF
2,SOUTH BEND,IN,46628,261000,1990-01-03,1990,120,ST JOSEPH,CORPORATION,PIF
3,HARRISBURG,IL,62946,262000,1990-01-03,1990,240,SALINE,CORPORATION,CHGOFF
6,SALT LAKE CITY,UT,84106,154000,1990-01-03,1990,240,SALT LAKE,CORPORATION,PIF
7,SALT LAKE CITY,UT,84104,135000,1990-01-03,1990,240,SALT LAKE,CORPORATION,PIF
8,COLUMBUS,OH,43215,300000,1990-01-03,1990,240,FRANKLIN,CORPORATION,PIF
9,DAYTON,OH,45427,284000,1990-01-04,1990,240,MONTGOMERY,CORPORATION,PIF
10,HOPEWELL,VA,23860,479000,1990-01-05,1990,240,HOPEWELL CITY,CORPORATION,PIF
11,EAU CLAIRE,WI,54701,750000,1990-01-05,1990,240,EAU CLAIRE,CORPORATION,PIF


In [9]:
# newdf.groupby('').size()
features = newdf.as_matrix(columns = ['BorrCity', 'BorrState', 'BorrZip', 'GrossApproval', 'ApprovalDate',\
                                      'ApprovalFiscalYear', 'TermInMonths', 'ProjectCounty', 'BusinessType'])
labels = newdf.as_matrix(columns = ['LoanStatus'])

# concatenate features into one vectors
X = []
for i in range(len(features)):
    temp = []
    for word in features[i]:
        temp.append(str(word))
    X.append(temp)

In [10]:
# use scipy.csr sparse matrix 
indices = []
data = []
indptr = [0]
vocab = {}
for row in X:
    for term in row:
        index = vocab.setdefault(term, len(vocab))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

X = csr_matrix((data, indices, indptr), dtype = int).toarray()
print 'Finished constructing CSR matrix for features... \nIt is a matrix of {}.'.format(X.shape)
print X[:10]

Finished constructing CSR matrix for features... 
It is a matrix of (54719, 27809).
[[1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [11]:
# get ground truth labels

_, idx = np.unique(labels, return_index=True)
gtlabels = labels[np.sort(idx)]

In [12]:
print gtlabels

[[u'PIF']
 [u'CHGOFF']]


In [13]:
# splitting data into train and test
ratio = 0.1
labels = np.reshape(labels, (len(labels), ))
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = ratio)
train_samples, n_features = x_train.shape
n_classes = len(gtlabels)

print train_samples, n_features
print n_classes

49247 27809
2


In [14]:
from sklearn.linear_model import LogisticRegression
solver = 'saga'

lr = LogisticRegression(solver=solver, 
                        multi_class= 'multinomial', 
                        C=1,
                        penalty = 'l1',
                        fit_intercept=True,
                        max_iter=10,
                        random_state=42, 
                       )
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)
np.sum(y_pred == y_test) *1./ y_test.shape[0]

# receiving convergence warning on data of 100000 - possible solutions are : 1. increase max iteraction (right now
# it is 10) or 2. data simply can't be fitted by a logistic model

/Users/hujiayu/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.85288742690058483

In [15]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)
np.mean(predicted == y_test)    

0.84009502923976609

In [16]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2',
              alpha=1e-3, random_state=42,
              max_iter=10, tol=None).fit(x_train, y_train)
predicted = clf.predict(x_test)
np.mean(predicted == y_test)    

0.85471491228070173

In [27]:
gtlabels = np.reshape(gtlabels, (2, ))

In [28]:
# take a look at precision vs. recall metrics

from sklearn import metrics

print metrics.classification_report(y_test, predicted,
                                    gtlabels)

             precision    recall  f1-score   support

        PIF       0.89      0.95      0.92      4586
     CHGOFF       0.58      0.38      0.46       886

avg / total       0.84      0.85      0.84      5472



In [29]:
# check for overfitting - see there is a large gap between train accuracy and test accuracy
predicted = clf.predict(x_train)
np.mean(predicted == y_train)  

0.86888541433995981